# Import necessary Libraries 

In [309]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np

# NLTK
import nltk
from nltk import TweetTokenizer, word_tokenize
from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# from nltk.stem import WordNetLemmatizer
# from nltk.tag import pos_tag
# from nltk import ne_chunk
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

# SPACY
import spacy
nlp = spacy.load("en_core_web_sm")

# Others
import re
import string
from string import punctuation
import time
from spellchecker import SpellChecker

# Self-created reference files
# from emoticon_unicodes import *
# from slangs import *
from preprocessing_steps import Preprocess

# Init stopwords
STOPWORDS_LIST = list(stopwords.words('english'))
PUNCTUATION_LIST = list(string.punctuation)
# Add variations not captured by string.punctuation
PUNCTUATION_LIST.append("“")
PUNCTUATION_LIST.append("”")
PUNCTUATION_LIST.append("’")
PUNCTUATION_LIST.append("…")

In [2]:
# data_df = pd.read_csv('tweets_compiled_2020_06_to_2021_02.csv', encoding='utf-8')
# data_df = pd.read_csv('tweets_compiled_2020_09_to_2021_02.csv', encoding='utf-8')
data_df = pd.read_csv('tweets_filtered_compiled_2020_09_to_2021_02.csv', encoding='utf-8')
# data_df = data_df.dropna(subset=['full_text']) # Drop nan values in "full_text" column
data_df.head(10)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,favorited,retweeted,possibly_sensitive,lang,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status,country
0,2020-09-01 04:25:47+00:00,1300651040526733314,1300651040526733312,We have a list of 200 kids that signed up for the backpack giveaway. I’m in need of hand sanitizer to include. If you’re willing to donate you can receive a receipt 🧾 for a tax deduction. I need by Friday September… https://t.co/9e6LQ1h0nx,False,"[0, 239]","{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/9e6LQ1h0nx', 'expanded_url': 'https://www.instagram.com/p/CElJeg4s3db/?igshid=1jjktp8i9ifji', 'display_url': 'instagram.com/p/CElJeg4s3db/…', 'indices': [216, 239]}]}","<a href=""http://instagram.com"" rel=""nofollow"">Instagram</a>",NaN,NaN,...,False,False,0.0,en,NaN,NaN,NaN,NaN,NaN,United States
1,2020-09-01 04:31:02+00:00,1300652360289660928,1300652360289660928,"Day 168 - Whenever you are struggling, you have to remember; “You are doing really well given the circumstances” #pandemic #thoughts @ Roosevelt Island https://t.co/8ZtRw7JTXx",False,"[0, 175]","{'hashtags': [{'text': 'pandemic', 'indices': [113, 122]}, {'text': 'thoughts', 'indices': [123, 132]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/8ZtRw7JTXx', 'expanded_url': 'https://www.instagram.com/p/CElKE0LDs9MReKE5N2NDm23zb40QDnfVnAiBNI0/?igshid=ewln0fasoikt', 'display_url': 'instagram.com/p/CElKE0LDs9MR…', 'indices': [152, 175]}]}","<a href=""http://instagram.com"" rel=""nofollow"">Instagram</a>",NaN,NaN,...,False,False,0.0,en,NaN,NaN,NaN,NaN,NaN,United States
2,2020-09-01 04:31:53+00:00,1300652576959008768,1300652576959008768,"#RedAlertRESTART: the live events we love may never recover from the pandemic, we need to take action! Take 2 minutes to contact your representatives here, and post a red photo of you at your favorite event, too:… https://t.co/7u056VcO1G",False,"[0, 237]","{'hashtags': [{'text': 'RedAlertRESTART', 'indices': [0, 16]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/7u056VcO1G', 'expanded_url': 'https://www.instagram.com/p/CElKLDnpPwK/?igshid=ig4keh0n6kxm', 'display_url': 'instagram.com/p/CElKLDnpPwK/…', 'indices': [214, 237]}]}","<a href=""http://instagram.com"" rel=""nofollow"">Instagram</a>",NaN,NaN,...,False,False,0.0,en,NaN,NaN,NaN,NaN,NaN,United States
3,2020-09-01 04:33:34+00:00,1300652998595551233,1300652998595551232,"Just golden sun, not another person on the river, and quarantine/mid-life crisis hair.\n\n#gayspeedo #rivertrash #midlifecrisis @ Pittsburg, California https://t.co/4Fv8rcwsIB",False,"[0, 173]","{'hashtags': [{'text': 'gayspeedo', 'indices': [88, 98]}, {'text': 'rivertrash', 'indices': [99, 110]}, {'text': 'midlifecrisis', 'indices': [111, 125]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/4Fv8rcwsIB', 'expanded_url': 'https://www.instagram.com/p/CElKXa0h4XI/?igshid=b5axcwrb2it7', 'display_url': 'instagram.com/p/CElKXa0h4XI/…', 'indices': [150, 173]}]}","<a href=""http://instagram.com"" rel=""nofollow"">Instagram</a>",NaN,NaN,...,False,False,0.0,en,NaN,NaN,NaN,NaN,NaN,United States
4,2020-09-01 04:35:11+00:00,1300653403706540034,1300653403706540032,🌟🌟[GIVEAWAY!]🌟🌟\n\nAre you still working from home but tired of eating the same old takeaways or spending too much on food deliveries? Here's another convenient option to consider - Fully cooked frozen products by… https://t.co/rmvLbYOvA0,False,"[0, 236]","{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/rmvLbYOvA0', 'expanded_url': 'https://www.instagram.com/p/CElKjF-H7e-/?igshid=tyj849ckzgbz', 'display_url': 'instagram.com/p/CElKjF-H7e-/…', 'indices': [213, 236]}]}","<a href=""http://instagram.com"" rel=""nofollow"">Instagram</a>",NaN,NaN,...,False,False,0.0,en,NaN,NaN,NaN,NaN,NaN,Singapore
5,2020-09-01

# Preprocessing Trial Codes

In [312]:
processed_txt = "Merry Christmas Fans😍 All we want for Christmas is Vaccine 🏩 @ Al Reem Island https://t.co/174bpYZlN3"
def convert_emoticons_to_words(text):
    processed_text = []
    tokenizer = TweetTokenizer()
    tokenized = tokenizer.tokenize(text)
    for token in tokenized: 
        # Convert emoticons to words
        if token in UNICODE_EMO:
            token = UNICODE_EMO[token].replace(":","")
        processed_text.append(token)
    return " ".join(processed_text)

# processed_txt = convert_emoticons_to_words(processed_txt)
# processed_txt

# processed_txt = Preprocess.convert_emoticons_to_words(processed_txt)
processed_txt = Preprocess.convert_to_lowercase(processed_txt)
processed_txt

'merry christmas fans😍 all we want for christmas is vaccine 🏩 @ al reem island https://t.co/174bpyzln3'

In [4]:
processed_txt = "Merry Christmas Fans😍 All we want for Christmas is Vaccine 🏩 @ Al Reem Island https://t.co/174bpYZlN3"

def convert_to_lowercase(text):
    return text.lower()

processed_txt = convert_to_lowercase(processed_txt)
processed_txt

'merry christmas fans😍 all we want for christmas is vaccine 🏩 @ al reem island https://t.co/174bpyzln3'

In [17]:
processed_txt = "Merry Christmas Fans😍 All we want for Christmas is Vaccine 🏩 @ Al Reem Island https://t.co/174bpYZlN3"

def remove_urls(text):
    regex_url_pattern = r"https?://\S+|www\.\S+"
    text = re.sub(regex_url_pattern, "", text)
    return text

processed_txt = remove_urls(processed_txt)
processed_txt

'Merry Christmas Fans😍 All we want for Christmas is Vaccine 🏩 @ Al Reem Island '

In [307]:
# processed_txt = "The Donald Trump is a \"famous\" US ex-president."
# processed_txt = "Can you recognise Barack Obama?"
# processed_txt = "How about San Franciso?"
# processed_txt = "Johnny Walker"
processed_txt = "Mannfred Von Carstein is a vampire!"
# processed_txt = "Donald Trump's car is very heavy."
# processed_txt = "My state fair game stand skills stay on point ...  I’m  really sad we had to break our tradition this year of attending due to Covid-19... this year the (Ye Old Mill) was a top priority… https://t.co/SYxjG97wxt"
# processed_txt = "HAPPY MONSTER MONDAAAAAY!!!! Don’t miss my Int/Adv Hip Hop Dance Class #567Eat  7pm 2night at groove2musikstudios SPACE IS LIMITED!!! Register NOW to secure your spot ;) ALWAYS wear a MASK 😷 @ Groove2Musik… https://t.co/4IqH57mkii"
# processed_txt = "Given Great Customer Service in COVID-19 Mask Up Thanks for the Love and Support Auntie Pat❤️. Book: your appointment online Instagram Facebook Twitter Text Call or Message Me (843)226-3315. @ Bossdiamond Dreads By… https://t.co/ytxzb3cb52"
# processed_txt = "50K+ #Google searches 4 Kyle Rittenhouse 2h old https://t.co/dQiRG3ddML https://t.co/ztKWIWDiZ6 #Illinois 🌅 6:44 🌆 18:50 CDT \n #Covid19 \n#Illinois 09/22/2020 \n New CS 1531 14DCSΔ +9.3% \nNew D 29 14DDΔ +3.3% https://t.co/berewqmXgX"

def entity_recognition_handling(text):
    processed_txt = []
    tokenizer = TweetTokenizer()
    text_with_details = nlp(text)
#     print("\n---------------------------------------------------------------------------------------------------------------\n")
#     print(text_with_details)
#     print("\n---------------------------------------------------------------------------------------------------------------\n")
    for ent in text_with_details.ents:
        sub_text = ent.text
        
#         print("\nBEFORE: " + sub_text + " - " + ent.label_)
        
        # Spacy entity recognition does not handle punctuation well # e.g. the "(Ye Old Mill)" becomes "(Ye Old Mill"
        sub_text = remove_punctuation(sub_text)

        # Remove any articles that come before the entity
        # e.g. 'a', 'A', 'the', 'The', 'an', 'An'
        sub_text = re.sub(r'\b[tT]he\s\b', '', sub_text)
        sub_text = re.sub(r'\b[aA]\s\b', '',sub_text)
        sub_text = re.sub(r'\b[aAn]\s\b', '',sub_text)
    
        # Remove trailing 's from named entities e.g. "Donald Trump's" or  "Donald Trump 's" becomes "Donald Trump"
        sub_text = re.sub(r' \'s', '', sub_text)
        
#         print("AFTER: " + sub_text + " - " + ent.label_)
        
        concatenated_text = "_".join(sub_text.split())
#         text = re.sub(sub_text, concatenated_text, text)
        # used string replace instead of re.sub due to some python bug "nothing to repeat at position 0" for unknown symbols
        text = text.replace(sub_text, concatenated_text); 
    return text

processed_txt = entity_recognition_handling(processed_txt)
processed_txt

'Mannfred_Von_Carstein is a vampire!'

In [144]:
# corpus = data_df['full_text'].values.tolist()
# corpus = [re.sub(r'-', ' ', string) for string in corpus]
# docs = [nlp(text) for text in corpus]
# labels = [ent.label_ for ent in docs.ents]
# Counter(labels)

In [26]:
processed_txt = "   this sentence      has     a lot  white    spaces"
def remove_multiple_white_spaces(text):
    text = re.sub('\s+', ' ', text) # Remove multiple white spaces that are concatenated together
    text = re.sub('^\s', '', text) # Remove white space at the start of sentences
    return text

processed_txt = remove_multiple_white_spaces(processed_txt)
processed_txt

'this sentence has a lot white spaces'

In [27]:
processed_txt = "1 2 34 hi five!"

def remove_digits(text):
    regex_digits_pattern = r"[1234567890]"
    text = re.sub(regex_digits_pattern, '', text)
    text = remove_multiple_white_spaces(text)
    return text

processed_txt = remove_digits(processed_txt)
processed_txt

'hi five!'

In [6]:
processed_txt = "one minute brb"
def convert_slangs_to_words(text):
    processed_text = []
    tokenizer = TweetTokenizer()
    tokenized = tokenizer.tokenize(text)
    for token in tokenized: 
        # Convert emoticons to words
        if token in SLANGS:
            token = SLANGS[token]
        processed_text.append(token)
    return " ".join(processed_text)

processed_txt = convert_slangs_to_words(processed_txt)
processed_txt

'one minute be right back'

In [7]:
processed_txt = "spellin corectin"

def correct_spelling_mistakes(text):
    corrected_text = []
    tokenizer = TweetTokenizer()
    tokenized = tokenizer.tokenize(text)
    spellcheck = SpellChecker()
    misspelled_words = spellcheck.unknown(tokenized)
    for token in tokenized:
        if token in misspelled_words:
            corrected_text.append(spellcheck.correction(token))
        else:
            corrected_text.append(token)
    return " ".join(corrected_text)
        
processed_txt = correct_spelling_mistakes(processed_txt)
processed_txt

'split correcting'

In [202]:
processed_text = "Merry Christmas Fans😍 All we want for Christmas is Vaccine 🏩 @ Al Reem Island https://t.co/174bpYZlN3"
# processed_text = "We have a list of 200 kids that signed up for the backpack giveaway. I’m in need of hand sanitizer to include. If you’re willing to donate you can receive a receipt 🧾 for a tax deduction. I need by Friday September… https://t.co/9e6LQ1h0nx"

def replace_hyphens_or_underscores_with_spaces(text):
    regex_hyphens_or_underscores_pattern = r"[-|_]"
    text = re.sub(regex_hyphens_or_underscores_pattern, " ", text)  
    return text

def remove_stopwords(text):
    processed_text = []
    tokenizer = TweetTokenizer()
    tokenized = tokenizer.tokenize(text)
    for token in tokenized: 
        if token.lower() not in STOPWORDS_LIST:
            processed_text.append(token)
    return " ".join(processed_text)

def remove_punctuation(text):
    processed_text = []
#     tokenizer = TweetTokenizer()
#     tokenized = tokenizer.tokenize(text)
    tokenized = word_tokenize(text) # Tokenizes punctuation too
    for token in tokenized: 
        if token not in PUNCTUATION_LIST:
            processed_text.append(token)
    return " ".join(processed_text)

processed_text = convert_to_lowercase(processed_text)
processed_text = convert_emoticons_to_words(processed_text)
processed_text = replace_hyphens_or_underscores_with_spaces(processed_text)
processed_text = remove_stopwords(processed_text)
processed_text = remove_punctuation(processed_text)
processed_text

'merry christmas fans smiling face heart eyes want christmas vaccine love hotel al reem island https //t.co/174bpyzln3'

In [9]:
processed_text = "i don't dislike it #Honest. i am ok with it #HAPPY"
# processed_text = "#RedAlertRESTART: the live events we love may never recover from the pandemic, we need to take action! Take 2 minutes to contact your representatives here, and post a red photo of you at your favorite event, too:… https://t.co/7u056VcO1G"

def prepend_NOT_to_handle_negation(text):
    negation_words = ["n't", "not", "no", "never"]
    prepend_NOT = False
    processed_text = []
#     tokenized = word_tokenize(text) # Tokenizes and split #HAPPY into #, HAPPY
    # Use TweetTokenizer as we should not append "NOT_" to hashtags
    tokenizer = TweetTokenizer() # Tokenizes and maintains hashtags, i.e. maintains #HAPPY
    tokenized = tokenizer.tokenize(text)
    for i in range(len(tokenized)):
        token = tokenized[i]
        
        # Check "prepend_NOT" flag, i.e. 1 of the previous word was a logical negation
        # Therefore, prepend "NOT_" until it reaches a punctuation symbol
        # IMPORTANT: Must NOT remove punctuation before this function
        if prepend_NOT:
            if prepend_NOT and token in PUNCTUATION_LIST:
                prepend_NOT = False
            else:
                # Do not append "NOT_" to hashtags
                if not is_hashtag(token):                 
                    token = "NOT_" + token
        
        # Note: Case sensitive. Therefore "NOT_" should not be captured
        regex_negation_pattern = r"(" + negation_words[0] + ")$"
        if re.search(regex_negation_pattern, token) or token in negation_words:
            prepend_NOT = True
        
        processed_text.append(token)
    return " ".join(processed_text)

def is_hashtag(token):
    is_hashtag = False
    regex_hashtag_pattern = r"^#\w+"
    if re.match(regex_hashtag_pattern, token):
        is_hashtag = True
    return is_hashtag
        
processed_text = prepend_NOT_to_handle_negation(processed_text)
processed_text

"i don't NOT_dislike NOT_it #Honest . i am ok with it #HAPPY"

# Compiled Test

In [308]:
def process_text(text):
#     print(text)
#     print("\n---------------------------------------------------------------------------------------------------------------\n")
    text = entity_recognition_handling(text)
    
#     text = convert_to_lowercase(text)
#     text = remove_urls(text)
#     text = convert_emoticons_to_words(text)
#     text = replace_hyphens_or_underscores_with_spaces(text)
#     text = convert_slangs_to_words(text)
#     text = prepend_NOT_to_handle_negation(text)
# #     text = remove_stopwords(text)
#     text = remove_punctuation(text)
# #     text = correct_spelling_mistakes(text)
    return text
    
processed_df = data_df.copy()    
# processed_df = processed_df.head(10)
processed_df["processed_text"] = processed_df["full_text"].apply(process_text)
processed_df[["full_text", "processed_text"]]

,full_text,processed_text
0,We have a list of 200 kids that signed up for the backpack giveaway. I’m in need of hand sanitizer to include. If you’re willing to donate you can receive a receipt 🧾 for a tax deduction. I need by Friday September… https://t.co/9e6LQ1h0nx,We have a list of 200 kids that signed up for the backpack giveaway. I’m in need of hand sanitizer to include. If you’re willing to donate you can receive a receipt 🧾 for a tax deduction. I need by Friday_September… https://t.co/9e6LQ1h0nx
1,"Day 168 - Whenever you are struggling, you have to remember; “You are doing really well given the circumstances” #pandemic #thoughts @ Roosevelt Island https://t.co/8ZtRw7JTXx","Day 168 - Whenever you are struggling, you have to remember; “You are doing really well given the circumstances” #pandemic #thoughts @ Roosevelt Island https://t.co/8ZtRw7JTXx"
2,"#RedAlertRESTART: the live events we love may never recover from the pandemic, we need to take action! Take 2 minutes to contact your representatives here, and post a red photo of you at your favorite event, too:… https://t.co/7u056VcO1G","#RedAlertRESTART: the live events we love may never recover from the pandemic, we need to take action! Take 2_minutes to contact your representatives here, and post a red photo of you at your favorite event, too:… https://t.co/7u056VcO1G"
3,"Just golden sun, not another person on the river, and quarantine/mid-life crisis hair.\n\n#gayspeedo #rivertrash #midlifecrisis @ Pittsburg, California https://t.co/4Fv8rcwsIB","Just golden sun, not another person on the river, and quarantine/mid-life crisis hair.\n\n#gayspeedo #rivertrash #midlifecrisis @ Pittsburg, California https://t.co/4Fv8rcwsIB"
4,🌟🌟[GIVEAWAY!]🌟🌟\n\nAre you still working from home but tired of eating the same old takeaways or spending too much on food deliveries? Here's another convenient option to consider - Fully cooked frozen products by… https://t.co/rmvLbYOvA0,🌟🌟[GIVEAWAY!]🌟🌟\n\nAre you still working from home but tired of eating the same old takeaways or spending too much on food deliveries? Here's another convenient option to consider - Fully cooked frozen products by… https://t.co/rmvLbYOvA0
...,...,...
48943,"Call us. We’ll keep you safe. Always wear a mask and keep 6’ apart. #realtors #nationalassociationofrealtors #rauschrealtor #teamrausch #getrausch #goldenglobes2020 #bettercallrausch @ Boca Raton, Florida https://t.co/V5OVCiLrZW","Call us. We’ll keep you safe. Always wear a mask and keep 6’ apart. #realtors #nationalassociationofrealtors #rauschrealtor #teamrausch #getrausch #goldenglobes2020 #bettercallrausch @ Boca_Raton, Florida https://t.co/V5OVCiLrZW"
48944,Selangor Journal | Change in operating hours at Selangor’s Covid-19 assessment centres https://t.co/szkScRF1Fq https://t.co/mDXwL0SS57,Selangor Journal | Change in operating_hours at Selangor’s Covid-19 assessment centres https://t.co/szkScRF1Fq https://t.co/mDXwL0SS57
48945,How do I prepare my body for a vaccine? https://t.co/xmozBD6Fdg,How do I prepare my body for a vaccine? https://t.co/xmozBD6Fdg
48946,Updated: Incident on #SouthernStateParkway EB at Exit 15 - North Corona Avenue https://t.co/JT1qQAfMM1,Updated: Incident on #SouthernStateParkway_EB at Exit 15 - North Corona Avenue https://t.co/JT1qQAfMM1
